Puzzle
-----

https://thefiddler.substack.com/p/can-you-crack-the-roman-code  

**This Week’s Fiddler**

You are breaking into a vault that contains ancient Roman treasure. The vault is locked, and can be opened via a modern-day keypad. The keypad contains three numerical inputs, which are (of course) expressed using Roman numerals: “I,” “II,” and “III.”

It’s a good thing your accomplice was able to steal the numerical key code to the vault. Earlier in the day, they handed you this code on a scroll of paper. Once at the keypad, you remove the scroll from your pocket and unfurl it. It reads: “IIIIIIIIII.” That’s ten vertical marks, without any clear spacing between them.

With some quick mental arithmetic, you realize the combination to unlock the door could be anywhere from four digits long to 10 digits long. (Or is it IV digits to X digits?) How many distinct combinations are possible? If two combinations use the same numbers but in a different order, they are considered distinct.


**This Week’s Extra Credit**

Having successfully hacked your way through the first keypad, the door opens to reveal a second door with yet another keypad that has eight numerical inputs: “I,” “II,” “III,” “IV,” “V,” “VI,” “VII,” and “VIII.”

You were expecting this, which is why your accomplice had handed you a second scroll of paper. You unfurl this one as well, hoping they remembered to add spaces between the numbers.

No such luck. This paper reads: “IIIVIIIVIIIVIII.” That’s 15 characters in total. How many distinct combinations are possible for this second door?

Fiddler solution
----

*If there's anything I have learnt from doing Fiddlers and Riddlers, it is to look for recurrence relations. :)*

Let $k$ be the length of the sequence of I's, and let $i_k$ be the number of ways in which it can be interpreted.

The start (or end) of the string could be "I", "II" or "III", and those interpretations would leave us with a string of length $k-1$ .. $k-3$.

So, we get $i_k = i_{k-1} + i_{k-2} + i_{k-3}$.

And the initial few terms can be calculated by hand. $i_1=1, i_2=2, i_3=4$.

We'll code this up shortly, but it can also be found on OEIS as (one of) the "Tribonacci Numbers".

https://oeis.org/A000073

And from that, we can count that $i_10 = 274$, which is the solution to this Fiddler.

Solution to the Extra Credit
----

The situation is similar to the fiddler, but the recurrence is not so tidy because the number of combinations that are a valid prefix (or suffix) varies with the exact string. e.g. "IIV" can be interpreted as I+I+V or II+V, but not IIV (3?). So, let's write some code to do the counting.

In [3]:
from functools import lru_cache

# We don't need these functions, but Copilot suggested them, and they are so neat, so let's use them to make our code more general.
roman_numeral_map = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
@lru_cache
def roman_to_int(s: str) -> int:
    total = 0
    prev_value = 0
    for char in reversed(s):
        value = roman_numeral_map[char]
        if value < prev_value:
            total -= value
        else:
            total += value
        prev_value = value
    return total

@lru_cache
def int_to_roman(num: int) -> str:
    val = [
        1000, 900, 500, 400,
        100, 90, 50, 40,
        10, 9, 5, 4,
        1
    ]
    syms = [
        "M", "CM", "D", "CD",
        "C", "XC", "L", "XL",
        "X", "IX", "V", "IV",
        "I"
    ]
    roman_numeral = ''
    for i in range(len(val)):
        while num >= val[i]:
            roman_numeral += syms[i]
            num -= val[i]
    return roman_numeral

def all_roman_numerals(N: int) -> list:
    return [int_to_roman(i) for i in range(1, N + 1)]

In [10]:
@lru_cache
def count_ambiguous_intpretations(s: str, symbols) -> int:
    if s == "":
        return 1
    count = 0
    for symbol in symbols:  
        if s.startswith(symbol):
            count += count_ambiguous_intpretations(s[len(symbol):], symbols)
    print(f"count_ambiguous_intpretations('{s}', ...) = {count}")
    return count

In [11]:
def solve_fiddler():
    input = "I"*10
    symbols = all_roman_numerals(3)
    fsymbols = frozenset(symbols)
    result = count_ambiguous_intpretations(input, fsymbols)
    print(f"Number of ambiguous interpretations for '{input}' with {symbols}: {result}")

def solve_extra_credit():
    input = ("I" * 3 + "V") * 3 + "I" * 3
    symbols = all_roman_numerals(8)
    fsymbols = frozenset(symbols)
    result = count_ambiguous_intpretations(input, fsymbols)
    print(f"Number of ambiguous interpretations for '{input}' with {symbols}: {result}")

solve_fiddler()
solve_extra_credit()

count_ambiguous_intpretations('I', ...) = 1
count_ambiguous_intpretations('II', ...) = 2
count_ambiguous_intpretations('III', ...) = 4
count_ambiguous_intpretations('IIII', ...) = 7
count_ambiguous_intpretations('IIIII', ...) = 13
count_ambiguous_intpretations('IIIIII', ...) = 24
count_ambiguous_intpretations('IIIIIII', ...) = 44
count_ambiguous_intpretations('IIIIIIII', ...) = 81
count_ambiguous_intpretations('IIIIIIIII', ...) = 149
count_ambiguous_intpretations('IIIIIIIIII', ...) = 274
Number of ambiguous interpretations for 'IIIIIIIIII' with ['I', 'II', 'III']: 274
count_ambiguous_intpretations('I', ...) = 1
count_ambiguous_intpretations('II', ...) = 2
count_ambiguous_intpretations('III', ...) = 4
count_ambiguous_intpretations('VIII', ...) = 8
count_ambiguous_intpretations('IVIII', ...) = 12
count_ambiguous_intpretations('IIVIII', ...) = 20
count_ambiguous_intpretations('IIIVIII', ...) = 40
count_ambiguous_intpretations('VIIIVIII', ...) = 80
count_ambiguous_intpretations('IVIIIVIII'

Ta-Da. We are done. 

The fiddler answer - 274 - matches what we expected from OEIS.

From OEIS, $i_{15}$ would be 5768, and 4000 seems plausible, since the V's make some combinations of length 3 like IIV disallowed, and some other combinations of length 4 like VIII allowed.
But 4000 is such a round number, it deserves some more investigation.

----

How come 4000 ?
----

Let $v_k$ be the ways to interpret the rightmost k letters in the input string. We calculated $v_{15} = 4000$.

Looking at how numbers are generated in the calculation steps above, we circle between adding 2, 3 or 4 previous terms.

IIIV... can start with I, II or III.

IIV... can start with I or II.

IV... can start with I or IV.

VIII... can start with V, VI, VII or VIII.

So, we can write out:

$v_{15} = v_{14} + v_{13} + v_{12}$

$v_{14} = v_{13} + v_{12}$

$v_{13} = v_{12} + v_{11}$

$v_{12} = v_{11} + v_{10} + v_9 + v_8$

$v_{11} = v_{10} + v_9 + v_8$

Substituting backwards, we can get:

$v_{12} = v_{11} + v_{11} = 2 * v_{11}$

$v_{13} = 2 * v_{11} + v_{11} = 3 * v_{11}$

$v_{14} = 3 * v_{11} + 2 * v_{11} = 5 * v_{11}$

$v_{15} = 5 * v_{11} + 3 * v_{11} + 2 * v_{11} = 10 * v_{11}$

In short:

$v_{15} = 10 * v_{11}$

And so the numbers increase by a factor of 10 every time we add 4 more letters, i.e. add another "IIIV" to the start, getting counts of 40,400,4000 ...

*[ The exact mapping is 1:2:3:5:10 :: 40:80:120:200:400 . I confused myself a bit because the sequence also has a 1,2,4,8 at the start which looks different from 1,2,3,5. ]*





